In [1]:
train_end_date ="2016-04-23"
train_14days = "2016-04-10"

In [2]:
path_hcba = 's3n://ewe-meta-top/hcba_data_2015_16/*'
hcba = sqlCtx.read.format("com.databricks.spark.csv").options(header = True, inferschema = True, delimiter = ",",nullValue= 
'\\N')\
    .load(path_hcba)
        

In [3]:
hcba1 = hcba.drop("")

In [4]:
hcba2 = sqlCtx.read.parquet("s3n://ewe-meta-top/hcba_final_last_yr_may_july")
hcba3 = hcba2.select("HOTEL_ID","PARTNER_POS","DEVICE_TYPE","PARTNER_ORG","ACTIVITY_DATE","TRAFFIC","BOOKING_WINDOW","LOS","COST","GROSS_PROFIT",
                     "LOS_BUCKET","BW_BUCKET","BOOKING","region","MODEL_NAME","STAR_RATING_BUCKET","HOTEL_TYPE_BUCKET","MARKET_BUCKET",\
                     "REGION_BUCKET","CITY_BUCKET",)

In [5]:
hcba4 = hcba1.unionAll(hcba3).repartition(1)

In [6]:
hcba_rm_gp0_book0 = (hcba4.filter((hcba4.GROSS_PROFIT !=0.00) & (hcba4.BOOKING != 0) & \
                                 (hcba4.ACTIVITY_DATE <= train_end_date) & (hcba4.DEVICE_TYPE=='DESKTOP') )
                     .withColumn("EPB",(hcba4.GROSS_PROFIT/hcba4.BOOKING))
                     .cache())

### Importing the data for all POS except CA

In [7]:
hcba_all1 = sqlCtx.read.parquet("s3n://ewe-meta-top/hcba_final_last_yr_may_march_allPOS")
hcba_all2 = hcba_all1.unionAll(sqlCtx.read.parquet("s3n://ewe-meta-top/hcba_final_this_yr_apr_june_allPOS1"))
hcba_all = hcba_all2.select("HOTEL_ID","PARTNER_POS","DEVICE_TYPE","PARTNER_ORG","ACTIVITY_DATE","TRAFFIC","BOOKING_WINDOW","LOS","COST","GROSS_PROFIT",
                     "LOS_BUCKET","BW_BUCKET","BOOKING","region","MODEL_NAME","STAR_RATING_BUCKET","HOTEL_TYPE_BUCKET","MARKET_BUCKET",\
                     "REGION_BUCKET","CITY_BUCKET")

In [8]:
hcba_all_rm = (hcba_all.filter((hcba_all.GROSS_PROFIT !=0.00) & (hcba_all.BOOKING != 0) & \
                                 (hcba_all.ACTIVITY_DATE <= train_end_date) & (hcba_all.DEVICE_TYPE=='DESKTOP') )
               .withColumn("EPB",(hcba_all.GROSS_PROFIT/hcba_all.BOOKING))
               .repartition(1)
               .cache()
               )

## UDF for Median

In [15]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

def cnt_n(count):
    if count%2 != 0:
        z = int(count/2)+1
        return z
    else : return int(count/2)
        
def bool(count,cnt_new,row_num):
    if count%2 != 0:
        if row_num == cnt_new:
            return 1
    else :
        if (row_num == cnt_new) | (row_num == cnt_new+1):
            return 1
        
bool_udf = udf(bool,IntegerType())  
cnt_udf = udf(cnt_n,IntegerType())


def median_func(df,cols_agg):
    
    from pyspark.sql.functions import rowNumber
    from pyspark.sql.window import Window

    w =  Window.partitionBy(cols_agg).orderBy(df.EPB)
    df1 = df.select("*","EPB",rowNumber().over(w).alias("row_num"))
    temp = df1.groupBy(cols_agg).count()
    temp1 = (df1.join(temp,cols_agg,"inner")
             .withColumn("cnt_new",cnt_udf("count"))
             .withColumn("bool_col",bool_udf("count","cnt_new","row_num"))
             .filter("bool_col = 1")
             .groupBy(cols_agg)
             .agg({"EPB":"avg"})
             .withColumnRenamed("avg(EPB)","epb"))
    
    return temp1

    

### EPB using recent 14 days and agg at hotel_id , los  and bw level

In [16]:
test1 =hcba_rm_gp0_book0.filter((hcba_rm_gp0_book0.ACTIVITY_DATE <= train_end_date) & (hcba_rm_gp0_book0.ACTIVITY_DATE >= train_14days))
test2 = median_func(test1,["HOTEL_ID","BW_BUCKET","LOS_BUCKET"])

In [17]:
from pyspark.sql.functions import *
hcba_epb1 = test2.withColumn("EPB1",round("epb",2)).drop("epb").repartition(1).cache()

In [18]:
test1 = hcba_all_rm.filter((hcba_all_rm.ACTIVITY_DATE <= train_end_date) & (hcba_all_rm.ACTIVITY_DATE >= train_14days))
test2 = median_func(test1,["HOTEL_ID","BW_BUCKET","LOS_BUCKET"])

In [19]:
hcba_epb_all1 = test2.withColumn("EPB_all1",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using data before 14 days and only this year and agg at hotel_id , los and bw level

In [20]:
test1 = hcba_rm_gp0_book0.filter((hcba_rm_gp0_book0.ACTIVITY_DATE < train_14days) & (hcba_rm_gp0_book0.ACTIVITY_DATE >='2016-01-01'))               
test2 = median_func(test1,["HOTEL_ID","BW_BUCKET","LOS_BUCKET"])

In [21]:
hcba_epb2 = test2.withColumn("EPB2",round("epb",2)).drop("epb").repartition(1).cache()

In [22]:
test1 = hcba_all_rm.filter((hcba_all_rm.ACTIVITY_DATE < train_14days) & (hcba_all_rm.ACTIVITY_DATE>='2016-01-01'))
test2 = median_func(test1,["HOTEL_ID","BW_BUCKET","LOS_BUCKET"])

In [23]:
hcba_epb_all2 = test2.withColumn("EPB_all2",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using recent 14 days data and agg at hotel_id and los level

In [24]:
test1 = hcba_rm_gp0_book0.filter((hcba_rm_gp0_book0.ACTIVITY_DATE <= train_end_date) & (hcba_rm_gp0_book0.ACTIVITY_DATE >=train_14days))
test2 = median_func(test1,["HOTEL_ID","LOS_BUCKET"])

In [25]:
hcba_epb3 = test2.withColumn("EPB3",round("epb",2)).drop("epb").repartition(1).cache()

In [26]:
test1 = hcba_all_rm.filter((hcba_all_rm.ACTIVITY_DATE <= train_end_date) & (hcba_all_rm.ACTIVITY_DATE >=train_14days))
test2 = median_func(test1,["HOTEL_ID","LOS_BUCKET"])

In [27]:
hcba_epb_all3 = test2.withColumn("EPB_all3",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using data before 14 days and only this year and agg at hotel_id and los level

In [28]:
test1 = hcba_rm_gp0_book0.filter((hcba_rm_gp0_book0.ACTIVITY_DATE < train_14days) & (hcba_rm_gp0_book0.ACTIVITY_DATE>='2016-01-01'))
test2 = median_func(test1,["HOTEL_ID","LOS_BUCKET"])

In [29]:
hcba_epb4 = test2.withColumn("EPB4",round("epb",2)).drop("epb").repartition(1).cache()

In [30]:
test1 = hcba_all_rm.filter((hcba_all_rm.ACTIVITY_DATE < train_14days) & (hcba_all_rm.ACTIVITY_DATE>='2016-01-01'))
test2 = median_func(test1,["HOTEL_ID","LOS_BUCKET"])

In [31]:
hcba_epb_all4 = test2.withColumn("EPB_all4",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using  data of last year and agg at hotel_id ,los and bw level

In [32]:
test1 = hcba_rm_gp0_book0.filter(hcba_rm_gp0_book0.ACTIVITY_DATE < "2016-01-01")
test2 = median_func(test1,["HOTEL_ID","BW_BUCKET","LOS_BUCKET"])

In [33]:
hcba_epb5 = test2.withColumn("EPB5",round("epb",2)).drop("epb").repartition(1).cache()

In [34]:
test1 = hcba_all_rm.filter(hcba_all_rm.ACTIVITY_DATE < "2016-01-01")
test2 = median_func(test1,["HOTEL_ID","BW_BUCKET","LOS_BUCKET"])

In [35]:
hcba_epb_all5 = test2.withColumn("EPB_all5",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using data of last year and agg at hotel_id and los level

In [36]:
test1 = hcba_rm_gp0_book0.filter(hcba_rm_gp0_book0.ACTIVITY_DATE < "2016-01-01")
test2 = median_func(test1,["HOTEL_ID","LOS_BUCKET"])

In [37]:
hcba_epb51 = test2.withColumn("EPB51",round("epb",2)).drop("epb").repartition(1).cache()

In [38]:
test1 = hcba_all_rm.filter(hcba_all_rm.ACTIVITY_DATE < "2016-01-01")
test2 = median_func(test1,["HOTEL_ID","LOS_BUCKET"])

In [39]:
hcba_epb_all51 = test2.withColumn("EPB_all51",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using this year data and agg at hotel_id level

In [40]:
test1 = hcba_rm_gp0_book0.filter((hcba_rm_gp0_book0.ACTIVITY_DATE >= '2016-01-01') & (hcba_rm_gp0_book0.ACTIVITY_DATE <= train_end_date))
test2 = median_func(test1,["HOTEL_ID"])

In [41]:
hcba_epb6 = test2.withColumn("EPB6",round("epb",2)).drop("epb").repartition(1).cache()

In [42]:
test1 = hcba_all_rm.filter((hcba_all_rm.ACTIVITY_DATE >= '2016-01-01') & (hcba_all_rm.ACTIVITY_DATE <= train_end_date))
test2 = median_func(test1,["HOTEL_ID"])

In [43]:
hcba_epb_all6 = test2.withColumn("EPB_all6",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using last year data and agg at hotel_id level

In [44]:
test1 = hcba_rm_gp0_book0.filter(hcba_rm_gp0_book0.ACTIVITY_DATE < "2016-01-01")
test2 = median_func(test1,["HOTEL_ID"])

In [45]:
hcba_epb7 = test2.withColumn("EPB7",round("epb",2)).drop("epb").repartition(1).cache()

In [46]:
test1 = hcba_all_rm.filter(hcba_all_rm.ACTIVITY_DATE < "2016-01-01")
test2 = median_func(test1,["HOTEL_ID"])

In [47]:
hcba_epb_all7 = test2.withColumn("EPB_all7",round("epb",2)).drop("epb").repartition(1).cache()

## joining with hotel_dim table to get market and other fields

In [48]:
path_hotel_dim = 's3n://ewe-core-meta-prod/CORE/HOTEL_DIM/*'
hotel_dim = sqlCtx.read.format("com.databricks.spark.csv").options(header = False, inferSchema = True, delimiter = "\t", nullValue= '\\N').			load(path_hotel_dim)
hotel_dim1 = hotel_dim.distinct().filter("C12 = 'BOOKABLE' or C12 ='NOT BOOKABLE' or C12 ='NOT APPLICABLE'").\
                select("C0","C7","C25","C26","C4","C13","C3")
hotel_dim4 = hotel_dim1.withColumnRenamed("C0","hotel_id").withColumnRenamed("C7" , "city").withColumnRenamed("C25","region").\
    withColumnRenamed("C26","market").withColumnRenamed("C4","brand").withColumnRenamed("C13","star_rating").\
    withColumnRenamed("C3","hotel_type")

#removing the records with hotel id 0
hotel_dim5 = hotel_dim4.filter((hotel_dim4['hotel_id'].isNotNull())).withColumnRenamed('hotel_id','HOTEL_ID').repartition(1).cache()


In [49]:
hcba_rm_gp0_book0_dim = hotel_dim5.join(broadcast(hcba_rm_gp0_book0),"HOTEL_ID","inner")

In [50]:
hcba_all_rm_dim = hcba_all_rm.join(hotel_dim5,"HOTEL_ID","inner").repartition(1).cache()

### EPB using this year data and agg at Market,Star_Rating,Hotel_Type,los bucket level

In [51]:
test1 = hcba_rm_gp0_book0_dim.filter("ACTIVITY_DATE >= '2016-01-01'").filter(hcba_rm_gp0_book0_dim.ACTIVITY_DATE <= train_end_date)
test2 = median_func(test1,["market","star_rating","hotel_type","LOS_BUCKET"])

In [52]:
hcba_epb8 = test2.withColumn("EPB8",round("epb",2)).drop("epb").repartition(1).cache()

In [53]:
test1 = hcba_all_rm_dim.filter("ACTIVITY_DATE >= '2016-01-01'").filter(hcba_all_rm_dim.ACTIVITY_DATE <= train_end_date)
test2 = median_func(test1,["market","star_rating","hotel_type","LOS_BUCKET"])

In [54]:
hcba_epb_all8 = test2.withColumn("EPB_all8",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using last year data and agg at Market,Star_Rating,Hotel_Type,los bucket level

In [55]:
test1 = hcba_rm_gp0_book0_dim.filter("ACTIVITY_DATE < '2016-01-01'")
test2 = median_func(test1,["market","star_rating","hotel_type","LOS_BUCKET"])


In [56]:
hcba_epb9 = test2.withColumn("EPB9",round("epb",2)).drop("epb").repartition(1).cache()

In [57]:
test1 = hcba_all_rm_dim.filter("ACTIVITY_DATE < '2016-01-01'  ")
test2 = median_func(test1,["market","star_rating","hotel_type","LOS_BUCKET"])

In [58]:
hcba_epb_all9 = test2.withColumn("EPB_all9",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using this year data and agg at Market,Star_Rating,Hotel_Type level

In [59]:
test1 = hcba_rm_gp0_book0_dim.filter("ACTIVITY_DATE >= '2016-01-01'").filter(hcba_rm_gp0_book0_dim.ACTIVITY_DATE <= train_end_date)

test2 = median_func(test1,["market","star_rating","hotel_type"])

In [60]:
hcba_epb10 = test2.withColumn("EPB10",round("epb",2)).drop("epb").repartition(1).cache()

In [61]:
test1 = hcba_all_rm_dim.filter("ACTIVITY_DATE >= '2016-01-01'").filter(hcba_all_rm_dim.ACTIVITY_DATE <= train_end_date)

test2 = median_func(test1,["market","star_rating","hotel_type"])

In [62]:
hcba_epb_all10 = test2.withColumn("EPB_all10",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using last year data and agg at Market,Star_Rating,Hotel_Type level

In [63]:
test1 = hcba_rm_gp0_book0_dim.filter("ACTIVITY_DATE < '2016-01-01'  ")

test2 = median_func(test1,["market","star_rating","hotel_type"])

In [64]:
hcba_epb11 = test2.withColumn("EPB11",round("epb",2)).drop("epb").repartition(1).cache()

In [65]:
test1 = hcba_all_rm_dim.filter("ACTIVITY_DATE < '2016-01-01'  ")

test2 = median_func(test1,["market","star_rating","hotel_type"])

In [66]:
hcba_epb_all11 = test2.withColumn("EPB_all11",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using this year data and agg at Market level

In [67]:
test1 =  hcba_rm_gp0_book0_dim.filter("ACTIVITY_DATE >= '2016-01-01'").filter(hcba_rm_gp0_book0_dim.ACTIVITY_DATE <= train_end_date)

test2 = median_func(test1,["market"])

In [68]:
hcba_epb12 = test2.withColumn("EPB12",round("epb",2)).drop("epb").repartition(1).cache()

In [69]:
test1 =  hcba_all_rm_dim.filter("ACTIVITY_DATE >= '2016-01-01'").filter(hcba_all_rm_dim.ACTIVITY_DATE <= train_end_date)

test2 = median_func(test1,["market"])

In [70]:
hcba_epb_all12 = test2.withColumn("EPB_all12",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using last year data and agg at Market level

In [71]:
test1 = hcba_rm_gp0_book0_dim.filter("ACTIVITY_DATE < '2016-01-01'  ")
test2 = median_func(test1,["market"])

In [72]:
hcba_epb13 = test2.withColumn("EPB13",round("epb",2)).drop("epb").repartition(1).cache()

In [73]:
test1 = hcba_all_rm_dim.filter("ACTIVITY_DATE < '2016-01-01'  ")
test2 = median_func(test1,["market"])

In [74]:
hcba_epb_all13 = test2.withColumn("EPB_all13",round("epb",2)).drop("epb").repartition(1).cache()

### creating universe for optimization

In [75]:
from datetime import timedelta, date
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import *
import time

##fucntion for chaging the datatype
def data_type_change_int(df,cols):
    temp = df
    for col in cols:
        temp = temp.withColumn(col+'_1',df[col].cast(IntegerType())).drop(col).withColumnRenamed(col+'_1',col)
    return temp

def data_type_change_float(df,cols):
    temp = df
    for col in cols:
        temp = temp.withColumn(col+'_1',df[col].cast(DoubleType())).drop(col).withColumnRenamed(col+'_1',col)
    return temp

#star rating buckets
def srch_region_star_rating(star_rating):
    if star_rating == '2.0':
        return '2'
    elif star_rating == '2.5':
        return '2.5'
    elif star_rating == '3.0':
        return '3'
    elif star_rating == '3.5':
        return '3.5'
    elif star_rating == '4.0':
        return '4'
    elif star_rating == '1.0' or star_rating == '1.5':
        return '1-1.5'
    elif star_rating == '4.5' or star_rating == '5.0':
        return '4.5-5'
    else:
        return '0'

#udf for creating city buckets
def srch_region_city(city,model_name):
	if model_name == 'CANADA':
		if city in ('TORONTO' , 'NIAGARA FALLS' , 'MONTREAL' , 'VANCOUVER' , 'QUEBEC' , 'BANFF' , 'OTTAWA' , 'EDMONTON' , 'VICTORIA','CALGARY' , 'WHISTLER'):
			return city
		elif city in ('NIAGARA-ON-THE-LAKE','CANMORE','JASPER','KELOWNA','HALIFAX','MISSISSAUGA','LAKELOUISE','COLLINGWOOD','VERNON','PENTICTON','GATINEAU','DORVAL','MARKHAM'):
			return'G12'
		elif city in ('MONT-TREMBLANT','TOFINO','BLUEMOUNTAINS','OSOYOOS','UCLUELET','SAINT-SAUVEUR','WESTKELOWNA','LAMALBAIE','SAINTE-ADELE'):
			return'G13'
		elif city in ('WINNIPEG','LONDON','RICHMOND','BARRIE','REVELSTOKE','VAUGHAN','ORILLIA','BURNABY','SURREY'):
			return 'G14'
		else:
			return 'others'
	else:
		return 'NA'

#udf for hotel type
def srch_region_Hotel_type(Hotel_type,model_name):
	if model_name == 'CANADA':
		if Hotel_type in  ('HOTEL' , 'HOTEL RESORT' , 'MOTEL'):
			return Hotel_type
		elif Hotel_type in ('CONDOMINIUM RESORT' , 'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('LODGE' , 'BED & BREAKFAST' , 'HOSTEL/BACKPACKER ACCOMMODATION' , 'CABIN' , 'GUEST HOUSE'):
			return 'Budget_hotel'
		elif Hotel_type in ('APART-HOTEL', 'INN'):
			return 'Family_Premium'
		else:
			return 'Others'
	
	elif model_name == 'US':
		if Hotel_type in  ('HOTEL' , 'HOTEL RESORT' , 'MOTEL'):
			return Hotel_type
		elif Hotel_type in ('CONDOMINIUM RESORT' ,'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('LODGE' , 'BED & BREAKFAST' , 'HOSTEL/BACKPACKER ACCOMMODATION' , 'CABIN' , 'GUEST HOUSE'):
			return 'Budget_hotel'
		elif Hotel_type in ('APART-HOTEL' ,  'INN' , 'APARTMENT', 'VILLA', 'PRIVATE VACATION HOME'):
			return 'Family_Premium'
		else:
			return 'others'
		
	elif model_name == 'Mexico':
		if Hotel_type in  ('HOTEL' , 'HOTEL RESORT' , 'ALL-INCLUSIVE'):
			return Hotel_type
		elif Hotel_type in ('CONDOMINIUM RESORT' ,'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('LODGE' , 'BED & BREAKFAST' , 'HOSTEL/BACKPACKER ACCOMMODATION'):
			return 'Budget_hotel'
		elif Hotel_type in ('APART-HOTEL' ,'INN' , 'VILLA' , 'APARTMENT'):
			return 'Family_Premium'
		else:
			return 'others'
		
	elif model_name=="CARIBBEAN":
		if Hotel_type in ('HOTEL' , 'HOTEL RESORT' , 'ALL-INCLUSIVE') :
			return Hotel_type
		elif Hotel_type in ('CONDOMINIUM RESORT' ,'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('APART-HOTEL' ,'INN' , 'VILLA' , 'APARTMENT'):
			return 'Family_Premium'
		else:
			return 'others'
		
	elif model_name == 'Other Region':
		if Hotel_type in ('HOTEL' , 'HOTEL RESORT' , 'APART-HOTEL') :
			return Hotel_type
		elif Hotel_type in ('GUEST HOUSE' ,'BED & BREAKFAST', 'HOSTEL/BACKPACKER ACCOMMODATION', 'MOTEL' , 'HOSTAL (BUDGET HOTEL)'):
			return 'Budget_hotel'
		elif Hotel_type in ('APARTMENT','RESIDENCE','TOWNHOUSE','INN'):
			return 'Budget_hotel'
		else:
			return 'others'
	
	elif model_name=="North America":
		if Hotel_type in ('HOTEL' , 'MOTEL') :
			return Hotel_type
		elif Hotel_type in ('LODGE' ,  'BED & BREAKFAST' ,  'COTTAGE' ,  'GUEST HOUSE' ,  'CABIN'):
			return 'Budget_hotel'
		elif Hotel_type in ( 'APARTMENT' ,  'HOTEL RESORT' ,  'CONDO' ,  'INN' ,  'APART-HOTEL' ,  'CARAVAN PARK'):
			return 'Family_Premium'
		else:
			return 'others'
	else:
		return 'NOT APPLICABLE'

##udf for market bucket
def srch_region_market(market,model_name):
	if model_name == 'CANADA':
		if market in  ('TORONTO, ON, CAN' , 'NIAGARA FALLS, ON, CAN' , 'MONTREAL, QC, CAN' , 'BANFF AREA, AB, CAN' ,'VANCOUVER, BC, CAN' ,'QUEBEC CITY, QC, CAN' , 'EDMONTON, AB, CAN' , 'OTTAWA, ON, CAN' , 'OKANAGAN VALLEY, BC, CAN'):
			return market
		elif market in ('VICTORIA, BC, CAN' ,'CALGARY, AB, CAN' ,'JASPER AREA, AB, CAN'):
			return 'G10'
		elif market in ('WHISTLER, BC, CAN' ,'NIAGARA-ON-THE-LAKE, ON, CAN' ,'MONT TREMBLANT, QC, CAN' , 'UCLUELET, BC, CAN','TOFINO - UCLUELET, BC, CAN'):
			return 'G11'
		elif market in ('COLLINGWOOD, ON, CAN' , 'BC INTERIOR SKI, BC, CAN' ,'HALIFAX, NS, CAN','WINNIPEG, MB, CAN' ,'LONDON, ON, CAN' , 'KAMLOOPS, BC, CAN'):
			return 'G12'
		else:
			return 'others'
	
	elif model_name == 'US':
		return 'NA'
		
	elif model_name == 'Mexico':
		if market in  ('MEXICO - RIVIERA MAYA, PLAYA DEL CARMEN & TULUM' , 'MEXICO - CANCUN & ISLA MUJERES' ,'MEXICO - PUERTO VALLARTA, NUEVO VALLARTA & RIVIERA NAYARIT' , 'MEXICO - LOS CABOS' , 'CENTRAL AMERICA - COSTA RICA - GUANACASTE' ,'CENTRAL AMERICA - BELIZE' , 'CENTRAL AMERICA - COSTA RICA - PUNTARENAS' , 'CENTRAL AMERICA - COSTA RICA - OTHER' , 'MEXICO - COZUMEL''MEXICO - HUATULCO' , 'MEXICO - IXTAPA & ZIHUATANEJO' , 'CENTRAL AMERICA - HONDURAS' , 'CENTRAL AMERICA - PANAMA - OTHER') :
			return market
		elif market in ( 'CENTRAL AMERICA - COSTA RICA - SAN JOSE' , 'CENTRAL AMERICA - PANAMA - PANAMA CITY' ,'MEXICO - MEXICO CITY' ,'MEXICO - MEXICO CITY'						,'CENTRAL AMERICA - NICARAGUA','MEXICO - MAZATLAN','MEXICO - MANZANILLO','CENTRAL AMERICA - EL SALVADOR','MEXICO - ACAPULCO'):
			return 'hclt'
		elif market in ('CENTRAL AMERICA - NICARAGUA','MEXICO - MAZATLAN','MEXICO - MANZANILLO','CENTRAL AMERICA - EL SALVADOR',                        'MEXICO - ACAPULCO'):
			return 'lclt'
		else:
			return 'others'
		
	elif model_name=="CARIBBEAN":
		if market in ('PUNTA CANA, DOMINICAN REPUBLIC' , 'TURKS AND CAICOS' , 'MONTEGO BAY, JAMAICA' , 'ST. LUCIA' ,'OCHO RIOS, JAMAICA' ,                      'NEGRIL, JAMAICA' , 'BARBADOS'):
			return market
		elif market in ('ARUBA' , 'PUERTO PLATA, DOMINICAN REPUBLIC' , 'ANTIGUA AND BARBUDA'):
			return 'G8'
		elif market in ('NASSAU, BAHAMAS', 'DOMINICAN REPUBLIC ALL OTHER' , 'BERMUDA','BAHAMAS, OUT ISLANDS'):
			return 'G9'
		elif market in ('SAN JUAN, PUERTO RICO' ,'PARADISE ISLAND, BAHAMAS' ,'CAYMAN ISLANDS','SINT MAARTEN, DUTCH' ,'CURACAO','ST. MARTIN, FRENCH'):
			return 'G10'
		else:
			return 'others'
		
	elif model_name == 'Other Region':
		return 'NA'
	
	elif model_name=="North America":
		if  market in (  'GRAVENHURST - BRACEBRIDGE - HUNTSVILLE, ON, CAN','ONTARIO WEST, ON, CAN','ABBOTSFORD - CHILLIWACK, BC, CAN',                       'MAINE - COAST','ONTARIO EAST, ON, CAN','VANCOUVER ISLAND, BC, CAN','BURLINGTON, VT','CHARLOTTETOWN, PEI, CAN'): 
			return 'G1'
		elif market in ('ALBERTA EAST, AB, CAN','REGINA, SK, CAN','KINGSTON, ON, CAN','SASKATOON, SK, CAN','MONCTON, NB, CAN',                        'TRI-CITIES (KW-GUELPH-CAMBRIDGE), ON, CAN'): 
			return 'G2'
		elif market in ('WINDSOR, ON, CAN','RED DEER, AB, CAN','BRITISH COLUMBIA SOUTH, BC, CAN','GRAND FORKS, ND','SUDBURY, ON, CAN',                        'ONTARIO NORTH, ON, CAN','BANGOR, ME','HAMILTON - BRANTFORD, ON, CAN','BELLEVILLE - TRENTON - COBOURG, ON, CAN','FARGO, ND'): 
			return 'G3'
		elif market in ('LAKE PLACID, NY','PARKSVILLE, VANCOUVER ISLAND, BC, CAN','LAKE GEORGE, NY','OGUNQUIT, ME','EVERETT, WA',                        'COASTAL, BC, CAN','GROVE CITY, PA','BELLINGHAM, WA','NORTH CONWAY, NH','NEWFOUNDLAND OTHER, NL, CAN',                        'NOVA SCOTIA SOUTH, NS, CAN','PORTSMOUTH, NH'): 
			return 'G4'
		elif market in ('LETHBRIDGE, AB, CAN','FREDERICTON, NB, CAN','BRANDON, MB, CAN','ERIE, PA','NANAIMO, VANCOUVER ISLAND, BC, CAN',                        'SASKATCHEWAN SOUTH, SK, CAN','PRINCE GEORGE - QUESNEL, BC, CAN','MEDICINE HAT, AB, CAN','GRANDE PRAIRIE, AB, CAN',                        'ALBERTA NORTH, AB, CAN','DURHAM, ON, CAN','BRITISH COLUMBIA CENTRAL, BC, CAN','ONTARIO SOUTHWEST, ON, CAN',                        'SAULT STE. MARIE, ON, CAN','THUNDER BAY, ON, CAN','SAINT JOHN, NB, CAN','GREAT FALLS, MT','MINOT, ND (AREA)',                        'BRITISH COLUMBIA NORTH, BC, CAN','NORTH BAY, ON, CAN','PETERBOROUGH, ON, CAN','CAPE BRETON ISLAND, NS, CAN','ALBERTA WEST, AB, CAN',                        'HUDSON VALLEY - POUGHKEEPSIE, NY','FORT MCMURRAY, AB, CAN','NOVA SCOTIA NORTH, NS, CAN','WATERTOWN, NY','EDMUNSTON, NB, CAN',                        'OLYMPIC NATIONAL PARK AREA, WA','CEDAR CITY - BRYCE CANYON NATIONAL PARK, UT','SUMMERSIDE, PEI, CAN','SALEM, OR (AREA)','BUTTE - HELENA, MT'                        ,'BECKLEY, WV'): 
			return 'G5'
		elif market in ('SAGINAW, MI','WHITE MOUNTAINS, NH','FINGER LAKES NEW YORK','NEW YORK - WEST','NEW BRUNSWICK NORTH, NB, CAN',                        'SASKATCHEWAN NORTH, SK, CAN','MANITOBA SOUTH, MB, CAN','BAR HARBOR, ME','WASHINGTON NORTHWEST','VERMONT CENTRAL','ALBERTA SOUTH, AB, CAN',                        'WENATCHEE-LEAVENWORTH,WA','GANANOQUE, ON, CAN','PLATTSBURGH, (NY)','STRAITS OF MACKINAC, MI','SANDUSKY, OH','MANCHESTER, NH','ITHACA, NY',                        'MINNESOTA NORTHWEST','MISSOULA, MT (AREA)','YELLOWKNIFE, NT, CAN','PEI OTHER, PEI, CAN','BILLINGS, MT (AREA)','RAPID CITY - MOUNT RUSHMORE, SD',                        'ANDOVER, MA','YUMA, AZ','PORT HURON, MI','LANCASTER, PA','FORT LEE - PARAMUS, NJ','LEXINGTON, KY','MANITOBA NORTH, MB, CAN','ABERDEEN - OCEAN SHORES, WA',                        'CANADA TERRITORIES, YT, CAN','KNOXVILLE, TN','BURLINGTON - MOUNT VERNON, WA','FREDERICKSBURG, VA','ARIZONA SOUTHEAST',                        'WINDHAM COUNTY, VT','KENAI PENINSULA, AK','COLUMBIA, SC','CALIFORNIA DESERTS SOUTH'): 
			return 'G6'
		elif  market in ('ROME-UTICA, NY','SANDPOINT, ID (AREA)','MOAB - GREEN RIVER, UT','WHITEHORSE, YT, CAN','NEW YORK - NORTHEAST',                         'SARNIA, ON, CAN','AUGUSTA, ME','LAKE POWELL - GLEN CANYON NATIONAL PARK, AZ','NEW YORK - SOUTH','BISMARK, ND',                         'ALABAMA NORTH','ASHLAND - MEDFORD, OR','WASHINGTON NORTHEAST','THE BERKSHIRES, MA','EUGENE, OR (AREA)',                         'GRAND RAPIDS, MI','WASHINGTON SOUTHWEST','LAKE HAVASU CITY, AZ','SARATOGA, NY','WYTHEVILLE - VA',                         'ARCATA - EUREKA, CA','SOUTH CAROLINA EAST','CHATTANOOGA, TN','TOLEDO, OH','IDAHO FALLS, ID','WEST VIRGINIA SOUTH',                         'SPRINGFIELD, MA','OLYMPIA, WA','GETTYSBURG - PA'): 
			return 'G7'
		else:return 'Others'
	else:return 'NOT APPLICABLE'
		
def srch_model_name(region):
	if region == 'CANADA':
		return region
		
	elif region in ('FLORIDA (USA)','NEW YORK CITY','CALIFORNIA','GAMING (USA)','NORTHEAST (USA)','MOUNTAIN (USA)',                    'MIDWEST (USA)','CENTRAL (USA)','SF PACNW','ORLANDO','HAWAII'):
		return 'US'
		
	elif region in ("MEXICO - DOMESTIC REGION",'MEXICO - RESORTS REGION','CENTRAL AMERICA REGION'):
		return 'Mexico'
		
	elif region=="CARIBBEAN":
		return 'CARIBBEAN'
    
	elif region=="NORTH AMERICA REGIONAL TERRITORIES":
		return 'North America'
		
	else :
		return "Other Region"

def srch_region_bucket(region,model_name):
	if model_name == 'Other Region':
		if region in ('SPAIN & PORTUGAL' , 'ROME & ITALY RESORTS' , 'MID-ATLANTIC' , 'GREECE & TURKEY' , 'OCEANIA' , 'ITALY NORTH' ,                      'THAILAND' , 'MIDDLE EAST AND INDIAN OCEAN'):
			return region
		elif region in ('PARIS' , 'LONDON' , 'ENGLAND'):
			return 'G9'
		elif region in ('EUROPE REGIONAL TERRITORIES' , 'UK & IRELAND'):
			return 'G10'
		elif region in ('SOUTH AMERICA' , 'AFRICA AND EAST MED'):
			return 'G11'
		elif region in ('EASTERN EUROPE' , 'GERMANY, AUSTRIA & SWITZERLAND' , 'BENELUX' , 'FRANCE' , 'NORDIC'):
			return 'G12'
		elif region in ('JAPAN & MICRONESIA' , 'HONG KONG & MACAU' , 'SOUTHEAST ASIA EMERGING' , 'MAINLAND CHINA'):
			return 'G13'
		else:
			return 'others'
	else :
		return 'NA'


udfsrch_region_star_rating=udf(srch_region_star_rating, StringType())
udfsrch_region_city=udf(srch_region_city, StringType())
udfsrch_region_hotel_type=udf(srch_region_Hotel_type, StringType())
udfsrch_region_market=udf(srch_region_market, StringType())
udfsrch_model_name= udf(srch_model_name,StringType())
udfsrch_region_bucket = udf (srch_region_bucket,StringType())

hotel_dim_model_name = hotel_dim5.withColumn("MODEL_NAME", udfsrch_model_name("region"))
hotel_dim_bkts = (hotel_dim_model_name.withColumn("STAR_RATING_BUCKET", udfsrch_region_star_rating("star_rating")).
                  withColumn("HOTEL_TYPE_BUCKET", udfsrch_region_hotel_type("hotel_type","MODEL_NAME")).    
                  withColumn("MARKET_BUCKET",udfsrch_region_market("market","MODEL_NAME")).     
                  withColumn("REGION_BUCKET", udfsrch_region_bucket("region","MODEL_NAME"))      
                  .withColumn("CITY_BUCKET", udfsrch_region_city("city","MODEL_NAME"))
                  .repartition(1).
                 cache())



In [76]:
bw = ['0','1','2','3-4','5-7','8-14','15-20','21-30','31-40','41-60','61-90','91-120','121-180','181-240','>240']
los = ['0','1','2','3','4','5','6','7','8-14','>14']

cnt = 1 

for i in range(len(los)):
    for j in range(len(bw)):
        if cnt == 1:
            sq_univ = hotel_dim_bkts.withColumn("LOS_BUCKET",lit(los[i])).withColumn("BW_BUCKET",lit(bw[j]))
        else :
            temp = hotel_dim_bkts.withColumn("LOS_BUCKET",lit(los[i])).withColumn("BW_BUCKET",lit(bw[j]))
            sq_univ = sq_univ.unionAll(temp)
        
        cnt+= 1
            
sq_univ_re = sq_univ.repartition(4).cache()

In [77]:
sq_univ1 = sq_univ_re.join((hcba_epb1),["HOTEL_ID","BW_BUCKET","LOS_BUCKET"],"left_outer")

In [78]:
sq_univ2 = sq_univ1.join((hcba_epb2),["HOTEL_ID","BW_BUCKET","LOS_BUCKET"],"left_outer")

In [79]:
sq_univ11 = sq_univ2.join((hcba_epb_all1),["HOTEL_ID","BW_BUCKET","LOS_BUCKET"],"left_outer")

In [80]:
sq_univ21 = sq_univ11.join((hcba_epb_all2),["HOTEL_ID","BW_BUCKET","LOS_BUCKET"],"left_outer")

In [81]:
sq_univ3 = sq_univ21.join(hcba_epb3,["HOTEL_ID","LOS_BUCKET"],"left_outer")

In [82]:
sq_univ4 = sq_univ3.join(hcba_epb4,["HOTEL_ID","LOS_BUCKET"],"left_outer")

In [83]:
sq_univ31 = sq_univ4.join(hcba_epb_all3,["HOTEL_ID","LOS_BUCKET"],"left_outer")

In [84]:
sq_univ41 = sq_univ31.join(hcba_epb_all4,["HOTEL_ID","LOS_BUCKET"],"left_outer")

In [85]:
sq_univ5 = sq_univ41.join(hcba_epb5,["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],"left_outer")

In [86]:
sq_univ6 = sq_univ5.join(hcba_epb51,["HOTEL_ID","LOS_BUCKET"],"left_outer")

In [87]:
sq_univ51 = sq_univ6.join(hcba_epb_all5,["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],"left_outer")

In [88]:
sq_univ61 = sq_univ51.join(hcba_epb_all51,["HOTEL_ID","LOS_BUCKET"],"left_outer")

In [89]:
sq_univ7 = sq_univ61.join(hcba_epb6,["HOTEL_ID"],"left_outer")

In [90]:
sq_univ8 = sq_univ7.join(hcba_epb7,["HOTEL_ID"],"left_outer")

In [91]:
sq_univ71 = sq_univ8.join(hcba_epb_all6,["HOTEL_ID"],"left_outer")

In [92]:
sq_univ81 = sq_univ71.join(hcba_epb_all7,["HOTEL_ID"],"left_outer")

In [93]:
sq_univ9 = sq_univ81.join(hcba_epb8,["market","star_rating","hotel_type","LOS_BUCKET"],"left_outer")

In [94]:
sq_univ10 = sq_univ9.join(hcba_epb9,["market","star_rating","hotel_type","LOS_BUCKET"],"left_outer")

In [95]:
sq_univ91 = sq_univ10.join(hcba_epb_all8,["market","star_rating","hotel_type","LOS_BUCKET"],"left_outer")

In [96]:
sq_univ101 = sq_univ91.join(hcba_epb_all9,["market","star_rating","hotel_type","LOS_BUCKET"],"left_outer")

In [97]:
sq_univ11 = sq_univ101.join(hcba_epb10,["market","star_rating","hotel_type"],"left_outer")

In [98]:
sq_univ12 = sq_univ11.join(hcba_epb11,["market","star_rating","hotel_type"],"left_outer")

In [99]:
sq_univ111 = sq_univ12.join(hcba_epb_all10,["market","star_rating","hotel_type"],"left_outer")

In [100]:
sq_univ121 = sq_univ111.join(hcba_epb_all11,["market","star_rating","hotel_type"],"left_outer")

In [101]:
sq_univ_coal = (sq_univ121.withColumn("EPB",coalesce('EPB1','EPB2','EPB3','EPB4','EPB5','EPB51','EPB6','EPB7','EPB8','EPB9','EPB10','EPB11',
                'EPB_all1','EPB_all2','EPB_all3','EPB_all4','EPB_all5','EPB_all51','EPB_all6','EPB_all7','EPB_all8','EPB_all9',
                                                    'EPB_all10','EPB_all11')))
sq_univ_final= (sq_univ_coal.select([col for col in sq_univ_coal.columns if col not in 
                {'EPB1','EPB2','EPB3','EPB4','EPB5','EPB51','EPB6','EPB7','EPB8','EPB9','EPB10','EPB11',
                'EPB_all1','EPB_all2','EPB_all3','EPB_all4','EPB_all5','EPB_all51','EPB_all6','EPB_all7','EPB_all8','EPB_all9',
                 'EPB_all10','EPB_all1'} ])
               .cache())

In [102]:
sq_univ_final[sq_univ_final.EPB.isNull()].count()

6053250

In [103]:
sq_univ_final.count()

47513400

In [105]:
6053250/47513400

0.12740090164037934

In [106]:
sq_univ_final.write.parquet("s3n://ewe-meta-top/median_epb_universe/sq_univ_media_epb_set4")

In [ ]:
sq_univ_coal.cache()

In [ ]:
sq_univ_coal[sq_univ_coal.EPB1.isNull() & sq_univ_coal.EPB2.isNull() & sq_univ_coal.EPB3.isNull()\
            & sq_univ_coal.EPB4.isNull() & sq_univ_coal.EPB5.isNull() & sq_univ_coal.EPB51.isNull()\
            & sq_univ_coal.EPB6.isNull() & sq_univ_coal.EPB7.isNull() & sq_univ_coal.EPB8.isNull()\
            & sq_univ_coal.EPB9.isNull() & sq_univ_coal.EPB10.isNull() & sq_univ_coal.EPB11.isNull()
            & sq_univ_coal.EPB_all1.isNull() & sq_univ_coal.EPB_all2.isNull() & sq_univ_coal.EPB_all3.isNull()
            & sq_univ_coal.EPB_all4.isNull()  & sq_univ_coal.EPB_all5.isNull() & sq_univ_coal.EPB_all51.isNull()
            & sq_univ_coal.EPB_all6.isNull() & sq_univ_coal.EPB_all7.isNull() & sq_univ_coal.EPB_all8.isNull()
            & sq_univ_coal.EPB_all9.isNull() & sq_univ_coal.EPB_all10.isNull() & sq_univ_coal.EPB_all11.isNull()].count()

In [ ]:
#new one 
16002508/38691058

In [ ]:
#new one 
5139817/38691058

In [ ]:
old one
17252315/38685724

### saving all epb tables in s3

In [ ]:
#hcba_epb1.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb1.csv')

In [ ]:
'''
hcba_epb2.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb2.csv')
hcba_epb3.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb3.csv')
hcba_epb4.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb4.csv')
hcba_epb5.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb5.csv')
hcba_epb6.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb6.csv')
'''

In [ ]:
'''
hcba_epb7.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb7.csv')
hcba_epb8.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb8.csv')
hcba_epb9.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb9.csv')
hcba_epb10.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb10.csv')
hcba_epb11.write.format('com.databricks.spark.csv').save('s3n://ewe-meta-top/epb_tables1/hcba_epb11.csv')
'''

# EDA

In [ ]:
hcba1.select("ACTIVITY_DATE").distinct().show(400)

In [ ]:
hcba1.agg({"BOOKING":"sum","TRAFFIC":"sum"}).show()

In [ ]:
hcba1.describe("GROSS_PROFIT").show()

In [ ]:
hcba1[hcba1.GROSS_PROFIT==0.00].count()

In [ ]:
hcba1.count()

In [ ]:
hcba1[hcba1.BOOKING==0].count()

In [ ]:
hcba_epb1_temp = hcba1.groupBy("HOTEL_ID","LOS_BUCKET","BW_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})

In [ ]:
from pyspark.sql.functions import *

hcba_epb1_temp1 = hcba_epb1_temp.withColumn("EPB_1",(hcba_epb1_temp["sum(GROSS_PROFIT)"]/hcba_epb1_temp["sum(BOOKING)"])).drop("sum(GROSS_PROFIT)").\
                drop("sum(BOOKING)")
    
hcba_epb1=hcba_epb1_temp1.withColumn("EPB",round(hcba_epb1_temp1.EPB_1,2)).drop("EPB_1")

In [ ]:
hcba_epb1[hcba_epb1.EPB==0.00].count()

In [ ]:
hcba_epb1[hcba_epb1.EPB.isNull()].count()

In [ ]:
hcba_epb1[hcba_epb1.EPB.isNull()].show(10)

In [ ]:
hcba[hcba.GROSS_PROFIT==0.00].count()

In [ ]:
hcba[hcba.BOOKING==0].count()

In [ ]:
hcba_epb1.count()

In [ ]:
hcba_epb1_temp1 = hcba1.filter("ACTIVITY_DATE<'2016-06-07'").groupBy("HOTEL_ID","LOS_BUCKET","BW_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})

from pyspark.sql.functions import *

hcba_epb1_temp2 = hcba_epb1_temp1.withColumn("EPB_1",(hcba_epb1_temp1["sum(GROSS_PROFIT)"]/hcba_epb1_temp1["sum(BOOKING)"])).drop("sum(GROSS_PROFIT)").\
                drop("sum(BOOKING)")
test_epb=hcba_epb1_temp2.withColumn("EPB",round(hcba_epb1_temp2.EPB_1,2)).drop("EPB_1")

In [ ]:
valid = hcba1.filter("ACTIVITY_DATE>='2016-06-07'")

In [ ]:
test_epb

In [ ]:
test_epb[~test_epb.EPB.isNull()].join(valid,["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],"inner").count()

In [ ]:
valid.count()

In [ ]:
hcba_epb1_temp1 = hcba1.filter("ACTIVITY_DATE<'2016-06-07'").groupBy("HOTEL_ID","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})

from pyspark.sql.functions import *

hcba_epb1_temp2 = hcba_epb1_temp1.withColumn("EPB_1",(hcba_epb1_temp1["sum(GROSS_PROFIT)"]/hcba_epb1_temp1["sum(BOOKING)"])).drop("sum(GROSS_PROFIT)").\
                drop("sum(BOOKING)")
test_epb2=hcba_epb1_temp2.withColumn("EPB2",round(hcba_epb1_temp2.EPB_1,2)).drop("EPB_1")

In [ ]:
test_epb2[~test_epb2.EPB2.isNull()].join(valid,["HOTEL_ID","LOS_BUCKET"],"inner").count()

In [ ]:
v1 = valid.join(test_epb[~test_epb.EPB.isNull()],["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],"left_outer")
v2 = v1.join(test_epb2[~test_epb2.EPB2.isNull()],["HOTEL_ID","LOS_BUCKET"],"left_outer")

In [ ]:
v3 = v2.withColumn("EPB_final",coalesce("EPB","EPB2"))

In [ ]:
v3[~v3.EPB_final.isNull()].count()

In [ ]:
v3.count()

In [ ]:
v1[~v1.EPB.isNull()].count()

In [ ]:
v2[~v2.EPB2.isNull()].count()

In [ ]:
v2[v2.EPB2.isNull()][v2.EPB.isNull()].count()

In [ ]:
hcba_remove_gp0_book0 = hcba1.filter("GROSS_PROFIT !=0.00 and BOOKING != 0 and ACTIVITY_DATE <= 2016-06-06")